In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [52]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 3)  # I/p - 28*28*1, Ch - 16, O/p - 26*26*16 Param - 3*3*16 + 16 RF - 3
        self.bn1 = nn.BatchNorm2d(16) 
        self.dpt1 = nn.Dropout(0.1)  

        self.conv2 = nn.Conv2d(16, 20, 3)  # I/p - 26*26*16, Ch - 20, O/p - 24*24*20 Param - 3*3*16*20 + 20 RF - 5
        self.bn2 = nn.BatchNorm2d(20)
        self.dpt2 = nn.Dropout(0.1)  

        self.conv22 = nn.Conv2d(20,10,1)  # I/p - 24*24*20, Ch - 16, O/p - 24*24*10 Param - 3*3*16*16 + 16 RF - 10
        self.pool1 = nn.MaxPool2d(2, 2)   # I/p - 24*24*1 O/p - 12*12*16 RF - 14

        self.conv3 = nn.Conv2d(10, 16, 3)  # I/p - 12*12*16, Ch - 16, O/p - 10*10*16 Param - 3*3*16*16 + 16 RF - 12
        self.bn3 = nn.BatchNorm2d(16)
        self.dpt3 = nn.Dropout(0.1)  
        
        self.conv4 = nn.Conv2d(16, 16, 3)   # I/p - 10*10*16, Ch - 16, O/p - 8*8*16 Param - 3*3*16*16 + 16 RF - 14
        self.bn4 = nn.BatchNorm2d(16)
        self.dpt4 = nn.Dropout(0.1)  
        
        self.conv5 = nn.Conv2d(16, 16, 3)    # I/p - 8*8*16, Ch - 16, O/p - 6*6*16 Param - 3*3*16*16 + 16 RF - 16
        self.bn5 = nn.BatchNorm2d(16)
        self.dpt5 = nn.Dropout(0.1)  
        self.pool2 = nn.MaxPool2d(2, 2)  # I/p - 6*6*16 O/p - 3*3*16 RF - 32
        

        self.conv6 = nn.Conv2d(16, 32, 1)  # I/p - 3*3*16 O/p - 3*3*32 RF - 34
        self.conv7 = nn.Conv2d(32, 10, 3)  # I/p - 3*3*32 O/p - 1*1*10 RF - 36


    def forward(self,x):
        x = self.dpt1(F.relu(self.bn1(self.conv1(x))))
        x = self.dpt2(self.pool1(self.conv22(self.bn2(F.relu(self.conv2(x))))))
        x = self.dpt3(self.bn3(F.relu(self.conv3(x))))
        x = self.dpt4(self.bn4(F.relu(self.conv4(x))))
        x = self.pool2(self.dpt5(self.bn5(F.relu(self.conv5(x)))))
        x = F.relu(self.conv6(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [53]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 20, 24, 24]           2,900
       BatchNorm2d-5           [-1, 20, 24, 24]              40
            Conv2d-6           [-1, 10, 24, 24]             210
         MaxPool2d-7           [-1, 10, 12, 12]               0
           Dropout-8           [-1, 10, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           1,456
      BatchNorm2d-10           [-1, 16, 10, 10]              32
          Dropout-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           2,320
      BatchNorm2d-13             [-1, 16, 8, 8]              32
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [54]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [56]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [57]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    print('EPOCH: ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0500, Accuracy: 9849/10000 (98.4900%)

EPOCH:  2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0196, Accuracy: 9939/10000 (99.3900%)

EPOCH:  13


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0191, Accuracy: 9945/10000 (99.4500%)

EPOCH:  14


loss=0.0257862638682127 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.74it/s]



Test set: Average loss: 0.0172, Accuracy: 9943/10000 (99.4300%)

